## Imports

In [2]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

from bs4 import BeautifulSoup as BS
import cloudscraper
import os.path
import json

## Get airports and their connecting airports

In [17]:
airportsFile = open("utils/airportList.csv")
airports = airportsFile.readlines()
airports = airports[0].split(',')

366

## Get connecting airports

In [16]:
# We want to webscrape urls that look like this:
# https://www.flightsfrom.com/ATL/destinations?countries=US&durationFrom=44&durationTo=999
scraper = cloudscraper.create_scraper()
connections = {}
if not os.path.isfile("utils/outgoingDictionary.txt"):
    for airport in airports:
        url = "https://www.flightsfrom.com/" + airport + "/destinations?countries=US&durationFrom=44&durationTo=999"
        r = scraper.get(url)
        soup = BS(r.text, 'html.parser')
        outgoingAirports = soup.find_all(class_ = "airport-font-midheader destination-search-item")
        connections[airport] = []
        for outgoing in outgoingAirports:
            connections[airport].append(str(outgoing).split(" ")[-1][:3])
        if len(connections[airport]) == 0:
            print(airport)
            del connections[airport]
    json.dump(connections,open("utils/outgoingDictionary.txt","x"))
else:
    connections = json.load(open("utils/outgoingDictionary.txt"))
    
connections

{'BHM': ['DFW',
  'ORD',
  'ATL',
  'DEN',
  'MIA',
  'CLT',
  'IAH',
  'LAS',
  'MCO',
  'PHL',
  'DTW',
  'DCA',
  'TPA',
  'MDW',
  'BWI',
  'LGA',
  'HOU',
  'DAL'],
 'DHN': ['ATL'],
 'HSV': ['DFW',
  'ORD',
  'ATL',
  'DEN',
  'MIA',
  'CLT',
  'IAH',
  'LAS',
  'MCO',
  'IAD',
  'DTW',
  'DCA',
  'TPA'],
 'MOB': ['DFW', 'ATL', 'CLT', 'IAH'],
 'MGM': ['DFW', 'ATL', 'CLT', 'DCA'],
 'ANC': ['DFW',
  'ORD',
  'ATL',
  'DEN',
  'LAX',
  'EWR',
  'IAH',
  'MSP',
  'LAS',
  'PHX',
  'SFO',
  'SEA',
  'SLC',
  'PDX',
  'HNL',
  'OGG',
  'FAI',
  'KOA',
  'JNU',
  'OME',
  'ADQ',
  'OTZ',
  'PAE',
  'BRW',
  'ILI',
  'UNK',
  'SCC',
  'SDP',
  'CDB',
  'CDV',
  'ANI',
  'ADK',
  'DUT',
  'AKN',
  'HOM',
  'DLG',
  'KSM',
  'VDZ',
  'SNP',
  'BET'],
 'BET': ['ANC'],
 'CDV': ['ANC', 'YAK'],
 'SCC': ['ANC', 'BRW'],
 'DLG': ['ANC'],
 'FAI': ['ORD',
  'MSP',
  'SEA',
  'ANC',
  'GAL',
  'HSL',
  'FYU',
  'VEE',
  'RBY',
  'TAL',
  'DJN',
  'HKB',
  'AET',
  'KBC',
  'BTI',
  'LMA',
  'AKP',
  

## Create links to all flight searches

In [11]:
#We want to create urls that look like this:
# "https://www.google.com/travel/flights?q=one%20way%20Flights%20to%20SNA%20from%20ATL%20on%202023-09-13"
if not os.path.isfile("utils/flightLinks.csv"):
    f = open("utils/flightLinks.csv","x")
    for startAirport in connections.keys():
        for endAirport in connections[startAirport]:
            searchURL = "https://www.google.com/travel/flights?q=one%20way%20Flights%20to%20"+endAirport+"%20from%20"+startAirport+"%20on%20,"
            f.write(searchURL+"\n")
    f.close()

## Create Airport Integer Encodings

In [15]:
airport_to_int = {}
for i,airport in enumerate(airports):
    airport_to_int[airport] = i
json.dump(airport_to_int,open("utils/airportEncodings.txt","x"))